# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping dos Bairros
Neste notebook, realizarei a extração dos dados dos bairros em Limeira, SP. Coletarei as localizações dos bairros e a distância de cada da faculdade.

## Geolocalização

In [ ]:
from geopy.geocoders import Nominatim
import geopy

In [ ]:
geolocator = Nominatim(user_agent="EstadiaLimeiraApp")

In [ ]:
location = geolocator.geocode("MASP, São Paulo")

print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

## Webscraping - Extração Bairros

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [ ]:
url = 'https://www.guiamais.com.br/bairros/limeira-sp'
response = urlopen(url)
html_bairros_limeira = response.read()
soup_bairros_limeira = BeautifulSoup(html_bairros_limeira, 'html.parser')

In [ ]:
bairros = []

# Coletando os bairros
for bairro in soup_bairros_limeira.find('section', {'class': 'cities'}).find_all('a'):
  bairros.append(bairro.get_text())

bairros = pd.Series(bairros)
print(bairros)

0                     Anavec
1      Área Rural de Limeira
2            Bairro da Geada
3           Bairro dos Lopes
4           Bairro dos Pires
               ...          
247            Vila São João
248            Vila São Luiz
249           Vila São Roque
250               Vila Solar
251    Vila Teixeira Marques
Length: 252, dtype: object


In [ ]:
bairros_df = []
lats = []
longs = []

# Determinar a localização de cada bairro
for bairro in bairros:
  location = geolocator.geocode(f"{bairro}, Limeira, São Paulo, Brazil")
  if location is not None:
    bairros_df.append(bairro)
    lats.append(location.latitude)
    longs.append(location.longitude)

In [ ]:
# Juntando os dados em um Data Frame
df_bairros = pd.DataFrame(data=None, columns=['bairro', 'latitude', 'longitude'])
df_bairros['bairro'] = bairros_df
df_bairros['latitude'] = lats
df_bairros['longitude'] = longs

df_bairros.head()

,bairro,latitude,longitude
0,Boa Vista,-22.606206,-47.548925
1,Centro,-22.565492,-47.403798
2,Chácara Antonieta,-22.555338,-47.436975
3,Chácara São José,-22.581140,-47.397673
4,Doutor João Batista Levy,-22.580151,-47.404356


In [ ]:
num_bairros_inicial = len(df_bairros)
print(f'Foram encontrados {num_bairros_inicial} bairros inicialmente.')

Foram encontrados 43 bairros inicialmente.


### Verificar bairros importantes

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.2 MB/s 


In [ ]:
import unidecode

def insert_row_to_df(df, row):
  row_sr = pd.Series(row, index=['bairro', 'latitude', 'longitude'])

  row_df = pd.DataFrame([row_sr])
  df = pd.concat([row_df, df], ignore_index=True)
  return df

def checkBairro(df, bairros_to_check):
  bairros = df_bairros['bairro'].map(lambda x: unidecode.unidecode(x).lower()).unique()
  bairros_to_check = list(map(lambda x: unidecode.unidecode(x).lower(), bairros_to_check))
  checksResult = []

  for bairro in bairros_to_check:
    checksResult.append(bairro in bairros)

  checks = pd.Series(checksResult, index=bairros_to_check)
  return checks

In [ ]:
bairros_to_check = ['jardim nova italia', 
                    'jardim paulista', 
                    'jardim sao paulo', 
                    'jardim morro azul', 
                    'jardim esmeralda',
                    'jardim perola',
                    'chacara antonieta',
                    'cidade universitaria',
                    'jardim santa fe',
                    'jardim nossa sra. de fatima',
                    'vila santa josefa',
                    'jardim santa cecilia',
                    'vila esteves',
                    'vila santa rosalia',
                    'parque real',
                    'vila sao joao',
                    'vila mathias',
                    'vila anita',
                    'parque sao bento',
                    'parque centreville',
                    'jardim aquarius',
                    'vila rocha']

# Retorna False se o bairro não está no dataframe
series_bairros_check = checkBairro(df_bairros, bairros_to_check)
series_bairros_check

jardim nova italia             False
jardim paulista                False
jardim sao paulo                True
jardim morro azul              False
jardim esmeralda               False
jardim perola                  False
chacara antonieta               True
cidade universitaria           False
jardim santa fe                False
jardim nossa sra. de fatima    False
vila santa josefa               True
jardim santa cecilia            True
vila esteves                    True
vila santa rosalia             False
parque real                     True
vila sao joao                  False
vila mathias                   False
vila anita                     False
parque sao bento               False
parque centreville             False
jardim aquarius                False
vila rocha                     False
dtype: bool

In [ ]:
# Saber os bairros importantes que não estão no dataframe
series_bairros_check[series_bairros_check == False]

jardim nova italia             False
jardim paulista                False
jardim morro azul              False
jardim esmeralda               False
jardim perola                  False
cidade universitaria           False
jardim santa fe                False
jardim nossa sra. de fatima    False
vila santa rosalia             False
vila sao joao                  False
vila mathias                   False
vila anita                     False
parque sao bento               False
parque centreville             False
jardim aquarius                False
vila rocha                     False
dtype: bool

In [ ]:
num_bairros_to_add = len(series_bairros_check[series_bairros_check == False])
print(f'{num_bairros_to_add} bairros que não estão no dataset! Adicioná-los manualmente.')

16 bairros que não estão no dataset! Adicioná-los manualmente.


#### Inserindo os bairros manualmente

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim nova italia', -22.559177655613762, -47.42176126705203])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim paulista', -22.558299143228314, -47.42619275210369])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim morro azul', -22.5584169798876, -47.43045410500301])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim esmeralda', -22.564260676816346, -47.435532311315484])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim perola', -22.563011669111834, -47.438237176617655])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['cidade universitaria', -22.553112082481263, -47.433785919560094])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim santa fe', -22.565597998868245, -47.422697294285875])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim nossa sra. de fatima', -22.56550370773466, -47.426499662401234])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['vila santa rosalia', -22.576232119192287, -47.406309958879994])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['vila sao joao', -22.563403058144573, -47.41099287617916])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['vila mathias', -22.560899389858037, -47.41035742731719])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['vila anita', -22.556810220964934, -47.410568487120784])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['parque sao bento', -22.552056052471702, -47.41803295839295])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['parque centreville', -22.551580463658322, -47.42222150033637])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['jardim aquarius', -22.550608190589706, -47.41667575205869])

In [ ]:
df_bairros = insert_row_to_df(df_bairros, ['vila rocha', -22.577951583104493, -47.410891226604896])

In [ ]:
df_bairros.head()

,bairro,latitude,longitude
0,vila rocha,-22.577952,-47.410891
1,jardim aquarius,-22.550608,-47.416676
2,parque centreville,-22.551580,-47.422222
3,parque sao bento,-22.552056,-47.418033
4,vila anita,-22.556810,-47.410568


In [ ]:
num_bairros_final = len(df_bairros)
assert num_bairros_inicial + num_bairros_to_add == num_bairros_final

In [ ]:
print(f'O dataset contém {num_bairros_final} bairros.')
print(f'Dos quais, {num_bairros_inicial} bairros com geopy e {num_bairros_to_add} adicionados manualmente.')

O dataset contém 59 bairros.
Dos quais, 43 bairros com geopy e 16 adicionados manualmente.


### Adicionar distância dos bairros até a FT Unicamp

In [ ]:
import geopy.distance

# Localização da faculdade
coords_ft_unicamp = (-22.561883227911103, -47.42418079671464)
distances = []

# Calcular as distâncias dos bairros até a faculdade
for i in range(len(df_bairros)):
  coords_aux = df_bairros.iloc[i, 1:]
  dist = geopy.distance.vincenty(coords_ft_unicamp, coords_aux).km
  distances.append(dist)

In [ ]:
# Adicionar as distâncias no dataframe
df_bairros['distancia_unicamp'] = distances
df_bairros.head()

,bairro,latitude,longitude,distancia_unicamp
0,vila rocha,-22.577952,-47.410891,2.243706
1,jardim aquarius,-22.550608,-47.416676,1.467927
2,parque centreville,-22.551580,-47.422222,1.158567
3,parque sao bento,-22.552056,-47.418033,1.258614
4,vila anita,-22.556810,-47.410568,1.508559


In [ ]:
from google.colab import files

# Exportar os dados
df_bairros.to_csv(r'bairros.csv', index=False)
files.download('bairros.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>